In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from functions import *
import requests
from bs4 import BeautifulSoup


In [2]:
def link(url):
    html = requests.get(url, headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'})
    soup = BeautifulSoup(html.content, 'html.parser')
    product_links = []
    for i in soup.find_all('a', 'cmp-category__item-link'):
        if len(i.get('href')) > 10 and "https://www.mcdonalds.com" + i.get('href') not in product_links:
            product_links.append("https://www.mcdonalds.com" + i.get('href'))
    return product_links

In [3]:
full_menu_url = "https://www.mcdonalds.com/tw/zh-tw/full-menu.html"
full_menu_html = requests.get(full_menu_url, headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'})
full_menu_soup = BeautifulSoup(full_menu_html.content, 'html.parser')
category_link = []
for j in full_menu_soup.find_all('ul','cmp-category__links-section cmp-category__row'):
    for i in j.find_all('a','category-link'):
        if len(i.get('href')) > 10 and "https://www.mcdonalds.com" + i.get('href') not in category_link:
            category_link.append("https://www.mcdonalds.com" + i.get('href'))
category_link

['https://www.mcdonalds.com/tw/zh-tw/full-menu/breakfast.html',
 'https://www.mcdonalds.com/tw/zh-tw/full-menu/extra-value-meals.html',
 'https://www.mcdonalds.com/tw/zh-tw/full-menu/signature-series.html',
 'https://www.mcdonalds.com/tw/zh-tw/full-menu/side-order.html',
 'https://www.mcdonalds.com/tw/zh-tw/full-menu/happy-sharing-box.html',
 'https://www.mcdonalds.com/tw/zh-tw/full-menu/happy-meal.html',
 'https://www.mcdonalds.com/tw/zh-tw/full-menu/50-dollars-meal.html',
 'https://www.mcdonalds.com/tw/zh-tw/full-menu/mccafe.html',
 'https://www.mcdonalds.com/tw/zh-tw/full-menu/beverages.html',
 'https://www.mcdonalds.com/tw/zh-tw/full-menu/snacks.html']

In [4]:
all_product_link = {}
for i in category_link:
    name =  i.split("/")[-1][:-5]
    all_product_link[name] = link(i)
all_product_link

{'breakfast': ['https://www.mcdonalds.com/tw/zh-tw/product/egg-mcmuffin.html',
  'https://www.mcdonalds.com/tw/zh-tw/product/sausage-mcmuffin.html',
  'https://www.mcdonalds.com/tw/zh-tw/product/sausage-mcmuffin-with-egg.html',
  'https://www.mcdonalds.com/tw/zh-tw/product/mega-mcmuffin.html',
  'https://www.mcdonalds.com/tw/zh-tw/product/chicken-muffin-with-egg.html',
  'https://www.mcdonalds.com/tw/zh-tw/product/chicken-muffin.html',
  'https://www.mcdonalds.com/tw/zh-tw/product/muffin-with-vegetables.html',
  'https://www.mcdonalds.com/tw/zh-tw/product/egg-burger-with-chicken.html',
  'https://www.mcdonalds.com/tw/zh-tw/product/egg-burger.html',
  'https://www.mcdonalds.com/tw/zh-tw/product/egg-burger-with-sausage.html',
  'https://www.mcdonalds.com/tw/zh-tw/product/egg-burger-with-canadian-bacon.html',
  'https://www.mcdonalds.com/tw/zh-tw/product/hotcakes-with-sausage.html',
  'https://www.mcdonalds.com/tw/zh-tw/product/hotcakes-3-pieces.html',
  'https://www.mcdonalds.com/tw/zh-t

In [5]:
def nutritionFacts(category, product_link):
    all_data_nutritionFacts = []
    for m in product_link:
        extra_link = []
        chrome = newBrowser(m)
        element = WebDriverWait(chrome, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'cmp-nutrition-summary--secondary-table-without-allergens')))  
        page = chrome.page_source
        soup = BeautifulSoup(page,"html.parser")
        for i in soup.find_all('a','cmp-product-details-main__variations-size-anchor'):
            if "https://www.mcdonalds.com" + i.get('href') != m and "https://www.mcdonalds.com" + i.get('href') not in extra_link: 
                extra_link.append("https://www.mcdonalds.com" + i.get('href'))

        name =  m.split("/")[-1][:-5]
        print(name)
        nutritionFacts = [category,name]
        for i,j in enumerate(soup.find_all('td','value')):
            if i %2 == 1:   
                a = BeautifulSoup(str(j))
                nutritionFacts.append(float(a.find('span').text.strip().replace(',','')))
        all_data_nutritionFacts.append(nutritionFacts)
        if len(extra_link) != 0:
            for k in extra_link:
                chrome = newBrowser(k)
                element = WebDriverWait(chrome, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'cmp-nutrition-summary--secondary-table-without-allergens')))
                page = chrome.page_source
                soup = BeautifulSoup(page,"html.parser")
                name = k.split("/")[-1][:-5]
                print(name)
                nutritionFacts = [category,name]
                for i,j in enumerate(soup.find_all('td','value')):
                    if i %2 == 1:   
                        a = BeautifulSoup(str(j))
                        nutritionFacts.append(float(a.find('span').text.strip().replace(',','')))
                all_data_nutritionFacts.append(nutritionFacts)
    return all_data_nutritionFacts
    

In [6]:
all_data =[]
for i,j in all_product_link.items():
    all_data.extend(nutritionFacts(i,j))


egg-mcmuffin
sausage-mcmuffin
sausage-mcmuffin-with-egg
mega-mcmuffin
chicken-muffin-with-egg
chicken-muffin
muffin-with-vegetables
egg-burger-with-chicken
egg-burger
egg-burger-with-sausage
egg-burger-with-canadian-bacon
hotcakes-with-sausage
hotcakes-3-pieces
big-breakfast
grand-breakfast
bagel-burger-with-bacon
bagel-burger-with-cheese
plain-bagel
big-mac
double-cheese-burger
grilled-bbq-chicken-burger
mcchicken
chicken-mcnuggets-6-pieces
chicken-mcnuggets-10-pieces
spicy-chicken-filet-burger
chicken-mccrispy-wing-2-pieces
spicy-chicken-mccrispy-wing-2-pieces
chicken-mccrispy-2-pieces
spicy-chicken-mccrispy-2-pieces
filet-o-fish
mushroom-angus-beef-burger
blt-angus-beef-burger
blt-spicy-chicken-burger
blt-grilled-chicken-burger
caesar-salad-with-spicy-fried-chicken-filet
italian-salad-with-grilled-chicken-filet
blt-angus-beef-burger
blt-spicy-chicken-burger
blt-grilled-chicken-burger
mushroom-angus-beef-burger
caesar-salad-with-spicy-fried-chicken-filet
italian-salad-with-grilled-ch

In [10]:
all_data

[['breakfast', 'egg-mcmuffin', 290.0, 19.0, 11.0, 5.3, 0.2, 28.0, 1.9, 696.4],
 ['breakfast',
  'sausage-mcmuffin',
  316.0,
  16.0,
  16.0,
  7.2,
  0.2,
  27.0,
  1.6,
  710.0],
 ['breakfast',
  'sausage-mcmuffin-with-egg',
  389.0,
  24.0,
  20.0,
  8.8,
  0.2,
  28.0,
  1.6,
  781.3],
 ['breakfast',
  'mega-mcmuffin',
  509.0,
  32.0,
  30.0,
  13.0,
  0.2,
  28.0,
  1.6,
  1002.8],
 ['breakfast',
  'chicken-muffin-with-egg',
  419.0,
  21.0,
  20.0,
  5.3,
  0.0,
  39.0,
  2.1,
  821.1],
 ['breakfast',
  'chicken-muffin',
  346.0,
  14.0,
  16.0,
  3.8,
  0.0,
  38.0,
  2.1,
  749.8],
 ['breakfast',
  'muffin-with-vegetables',
  292.0,
  19.0,
  11.0,
  5.3,
  0.2,
  28.0,
  2.2,
  698.5],
 ['breakfast',
  'egg-burger-with-chicken',
  414.0,
  21.0,
  19.0,
  5.8,
  0.1,
  40.0,
  5.1,
  758.5],
 ['breakfast', 'egg-burger', 282.0, 14.0, 12.0, 4.0, 0.1, 30.0, 4.9, 432.2],
 ['breakfast',
  'egg-burger-with-sausage',
  404.0,
  22.0,
  22.0,
  7.7,
  0.1,
  31.0,
  5.1,
  655.7],
 ['

In [21]:
data = pd.DataFrame(columns=['Category','Name','Calories (Kcal)','Protein (g)','Total Fat (g)','Saturated Fat (g)','Trans Fat (g)','Total Carbohydrate (g)','Total Sugars (g)','Sodium (mg)'])
for i in range(len(all_data)):
    data1 = pd.DataFrame([all_data[i]],columns=['Category','Name','Calories (Kcal)','Protein (g)','Total Fat (g)','Saturated Fat (g)','Trans Fat (g)','Total Carbohydrate (g)','Total Sugars (g)','Sodium (mg)'])
    data = pd.concat([data,data1])
data

,Category,Name,Calories (Kcal),Protein (g),Total Fat (g),Saturated Fat (g),Trans Fat (g),Total Carbohydrate (g),Total Sugars (g),Sodium (mg)
0,breakfast,egg-mcmuffin,290.0,19.0,11.0,5.3,0.2,28.0,1.9,696.4
0,breakfast,sausage-mcmuffin,316.0,16.0,16.0,7.2,0.2,27.0,1.6,710.0
0,breakfast,sausage-mcmuffin-with-egg,389.0,24.0,20.0,8.8,0.2,28.0,1.6,781.3
0,breakfast,mega-mcmuffin,509.0,32.0,30.0,13.0,0.2,28.0,1.6,1002.8
0,breakfast,chicken-muffin-with-egg,419.0,21.0,20.0,5.3,0.0,39.0,2.1,821.1
...,...,...,...,...,...,...,...,...,...,...
0,snacks,ice-cream-cone-large,202.0,3.6,6.5,3.4,0.0,32.0,25.0,100.4
0,snacks,mcflurry,352.0,5.7,12.0,6.2,0.0,55.0,41.0,265.3
0,snacks,chicken-mcnuggets-4-pieces,180.0,10.0,12.0,2.8,0.0,8.7,0.0,285.1
0,snacks,chicken-mcnuggets-6-pieces,270.0,16.0,17.0,4.2,0.0,13.0,0.0,427.6


In [22]:
data.to_csv('All_Data_NutritionFacts.csv',index=False)